In [3]:
import json
import os

import googlemaps
import numpy as np
import pandas as pd

In [7]:
with open('./apikeys.json', 'r') as f:
    apikeys = json.load(f)

map_client = googlemaps.Client(key=apikeys['google_maps'])

In [2]:
data = pd.read_csv('./data/bikeshare_ridership_2017.csv')

trips = data.groupby(['route_id', 'station_id_from', 'station_id_to', 'lat_from', 'lon_from', 'lat_to', 'lon_to'])['trip_id'].nunique().reset_index()

In [3]:
raw_routes = []
with open(os.path.join('./data/bikeshare_routes.json'), 'r') as f:
    raw_routes = json.load(f)

In [4]:
df = trips[~trips['route_id'].isin([r['key'] for r in raw_routes])]

In [5]:
for i, r in tqdm(df.head(1400).iterrows(), total=df.head(1400).shape[0]):
    path = { 'key': r['route_id'] }

    if r['station_id_from'] == r['station_id_to']:
        path['routes'] = []
    else:
        path['routes'] = map_client.directions(
                            '{lat},{lon}'.format(lat=np.round(r['lat_from'], 6), lon=np.round(r['lon_from'], 6)), 
                            '{lat},{lon}'.format(lat=np.round(r['lat_to'], 6), lon=np.round(r['lon_to'], 6)), 
                            mode='bicycling'
                         )

    raw_routes.append(path)

100%|██████████| 45/45 [00:13<00:00,  3.90it/s]


In [6]:
with open('./data/bikeshare_routes.json', 'w') as f:
    json.dump(raw_routes, f)

In [7]:
trips[~trips['route_id'].isin([r['key'] for r in raw_routes])].shape

(0, 8)

In [8]:
# routes = []
# for r in raw_routes:
#     if len(r['routes']) < 1:
#         continue
        
#     directions = r['routes'][0]['legs'][0]
#     tmp = {
#         'route_id': r['key'],
#         'distance': directions['distance']['value'],
#         'duration': directions['duration']['value'],
#         'steps': []
#     }
    
#     for s in directions['steps']:
#         tmp['steps'].append([
#             s['start_location']['lat'],
#             s['start_location']['lng']
#         ])
        
#     tmp['steps'].append([
#         directions['steps'][-1]['end_location']['lat'],
#         directions['steps'][-1]['end_location']['lng']
#     ])
    
#     routes.append(tmp)
    
# routes = pd.DataFrame(routes)
# routes.to_csv('./data/bikeshare_routes.csv', index=False)